In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn import Parameter
import math
import numpy as np
import collections
from sklearn import metrics
import pandas as pd
from sklearn.mixture import GaussianMixture
from scipy.optimize import linear_sum_assignment as linear_assignment
from torchvision import datasets
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from torchvision import transforms
from tqdm import tqdm
import seaborn as sns
from scipy.stats import skew, kurtosis
from sklearn.model_selection import KFold
import torch.utils.data as data
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F

from VAE_gamma import BasicDataset, generate_data, VAE_gamma, pretrain_fit, train_model, cluster_acc

In [3]:
n=100
delta = 0.1

In [4]:
data_all = generate_data(n,dim=10, set_1 = [5, 1, 10], set_2 = [15, 2, 10], delta=0.1)

In [5]:
label_1 = torch.zeros(n).int()
label_2 = torch.ones(n).int()

label_all = torch.cat([label_1,label_2],dim=0)

train_dataset = BasicDataset(data_all, label_all)
test_dataset = BasicDataset(data_all, label_all)

batch_size = int(n/10)


dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dataloader_test = torch.utils.data.DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True)

In [6]:
final_list = {}
n_pre=5
monte_number = 1
for i in range(1, n_pre+1):      

    if torch.cuda.is_available():
        pre_model =  VAE_gamma(data_dim=10, latent_dim=2, device=torch.device('cuda:5'), encodeLayer=[200,100,30],
                               decodeLayer=[30,100,200], num_cluster=2).cuda(torch.device('cuda:5'))

    optimizer_auto = optim.Adam(pre_model.parameters(), lr=0.001) 
    scheduler = StepLR(optimizer_auto, step_size=20, gamma= 0.9)
    path_name = '/home/jupyter-heojw/Data_all/VAE-gamma/weight/simulation/pretrain'+'_'+str(i)+'.pth'
    pretrain_epoch= 20

    pretrain_fit(dataloader, pre_model, optimizer_auto,scheduler,path_name,pretrain_epoch,monte_number,2,device=torch.device('cuda:5'))

    if torch.cuda.is_available():
        model_train = VAE_gamma(data_dim=10, latent_dim=2, device=torch.device('cuda:5'), encodeLayer=[200,100,30],
                               decodeLayer=[30,100,200], num_cluster=2).cuda(torch.device('cuda:5'))
    saveWeightPath = path_name
    optimizer = optim.Adam(model_train.parameters(), lr=0.001)
    scheduler = StepLR(optimizer, step_size=20, gamma= 0.9)
    model_train.load_state_dict(torch.load(saveWeightPath))
    epoch_train = 10

    path_name_2 = '/home/jupyter-heojw/Data_all/VAE-gamma/weight/simulation/simulation_' + '_initial' + str(i) +  '.pth'

    loss = train_model(dataloader, dataloader_test, model_train, optimizer,scheduler,epoch_train,batch_size,monte_number,10,torch.device('cuda:5'),path_name_2)
    final_list[' initial:'+str(i) ] = loss


Accurasy: 90.0
ARI: 0.6382285926668068
***** Test Result: loss: 0.828 acc: 95.5000
Accurasy: 91.0
ARI: 0.6707993237470232
***** Test Result: loss: 1.266 acc: 95.5000
Accurasy: 91.0
ARI: 0.6707880193267941
***** Test Result: loss: 1.299 acc: 96.5000
Accurasy: 92.0
ARI: 0.7041424182252666
***** Test Result: loss: 1.234 acc: 96.0000
Accurasy: 92.0
ARI: 0.7041424182252666
***** Test Result: loss: 0.746 acc: 96.5000


In [8]:
final_loss = np.array(list(final_list.items()))[:,1].astype('float')
print(final_loss)

[0.82834107 1.26550001 1.29873805 1.23401604 0.7464927 ]


In [10]:
optim_model = np.where(final_loss==final_loss.min())[0][0] + 1 

path_name_best =  '/home/jupyter-heojw/Data_all/VAE-gamma/weight/simulation/simulation_' + '_initial' + str(optim_model) +  '.pth'

if torch.cuda.is_available():
    model =  VAE_gamma(data_dim=10, latent_dim=2, device=torch.device('cuda:5'), encodeLayer=[200,100,30],
                           decodeLayer=[30,100,200], num_cluster=2).cuda(torch.device('cuda:5'))

saveWeightPath = path_name_best

model.load_state_dict(torch.load(saveWeightPath))
print('\n ************************ sample: '+ str(n),'delta: '+ str(delta)+ '************************')
model.eval()
accuracy_list = []
test_loss_list = []    
test_loss_sum = 0.0
for perform in range(0,100):
    test_loss = 0.0
    test_running_correct = 0.0
    label_set_test = torch.tensor([],dtype=int).cuda(torch.device('cuda:5'))
    preds_set_test = torch.tensor([],dtype=int).cuda(torch.device('cuda:5'))
    for batch_idx, data in enumerate(dataloader_test):

        img, label = data 
        img = img.float()
        img = img.view(img.size(0), -1)

        if torch.cuda.is_available():
            img = img.cuda(torch.device('cuda:5'))
            label = label.cuda(torch.device('cuda:5')).float()

        recon_al,recon_be, logalpha, logbeta, z = model(img)
        monte_gamma = 0
        monte_recon_loss = 0
        for l in range(monte_number):
            z_l = model.reparametrize(logalpha, logbeta)
            m = nn.Softplus()
            recon_al_l = 1e-6 + m(model.decode(z_l)[:,:model.data_dim])
            recon_be_l = 1e-6 + m(model.decode(z_l)[:,model.data_dim:])
            monte_gamma += model.get_gamma(z_l,len(dataloader_test.dataset))
            monte_recon_loss += model.recon_loss(recon_al_l,recon_be_l, img)
        sgvb_gamma_test = monte_gamma/monte_number
        sgvb_loss_test = torch.mean(monte_recon_loss/monte_number 
                                    + model.kl_loss(sgvb_gamma_test, logalpha, logbeta,len(dataloader_test.dataset))) #+ 1*model.Loss_b(img,z, logalpha, logbeta, 2, latent_dim))

        _, preds = torch.max(sgvb_gamma_test,1)

        label_set_test = torch.cat((label_set_test,label)).int()
        preds_set_test = torch.cat((preds_set_test,preds)).int()
        test_loss += sgvb_loss_test.item() * img.size(0)


    label_set_test = label_set_test.cpu()
    preds_set_test = preds_set_test.cpu()

    test_acc = cluster_acc(preds_set_test,label_set_test) * 100/preds_set_test.shape[0]
    test_loss_sum += test_loss
    accuracy_list = np.append(accuracy_list, test_acc)
test_loss_list = np.append(test_loss_list, test_loss_sum/100)


print('***** Test Result: loss: {:.3f} acc: {:.4f}'
  .format( test_loss_list[0] / len(dataloader_test.dataset), np.max(accuracy_list)))


 ************************ sample: 100 delta: 0.1************************
***** Test Result: loss: 0.747 acc: 96.5000
